# Define forcing parameters

## Import libraries

First of all, we load the necessary pre-processing libraries such as:

+ `pyvista`
+ `xarray`
+ `jigsaw`
+ `meshio`


In [1]:
import os
import sys
import meshio
import meshplex
import jigsawpy
import numpy as np
import pandas as pd
import xarray as xr

from scipy import ndimage
from scipy import interpolate
from scipy.ndimage.filters import gaussian_filter

from time import process_time
from gospl._fortran import definegtin

import matplotlib
import pyvista as pv
import matplotlib.pyplot as plt

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Unstructured global mesh construction

We first define a folder where input files will be stored:

In [2]:
# Resolution of the region of interest in km
res_mesh = 25

input_path = "input"+str(res_mesh)+"km" 
if not os.path.exists(input_path):
    os.makedirs(input_path)

This example focus on the Gulf of Mexico and we specify the extent of the region of interest. 

:::{note}

We define a region which encapsulate the large basins which drains into the Gulf.

:::

In [3]:
# Longitudinal extent
gom_lon = [-135, -45]

# Latitudinal extent
gom_lat = [10, 80]

### `Jigsaw` processing folder

The `Jigsaw` library requires an initial mesh. In our case we work with global dataset and we define an `ellipsoid-grid` with a 1 degree resolution containing a single `variable` set to:

+ -100 outside the region of interest, 
+ 100 in the specified region and 
+ 0 on the border of the region (5 degree width).  

This grid is built with the `buildRegMesh` function defined below:

In [4]:
def buildRegMesh(region_lon, region_lat, outfile, glon=181, glat=91):
    '''
    Create a lon/lat regular grid with values set to -100 everywhere except
    within the specified region where it is set to 100 and within 5 degree
    surrounding the region where it is set to 0.
    
    :arg region_lon: longitudinal extent 
    :arg region_lat: latitudinal extent  
    :arg outfile: name of the output file
    :arg glon: number of points along the longitudes (default: 181)
    :arg glat: number of points along the latitudes (default: 91)
    
    '''
    lon = np.linspace(-180.0, 180, glon)
    lat = np.linspace(-90.0, 90, glat)
    zval = np.zeros((glon, glat))
    ds = xr.Dataset(
                {"z": (["longitude", "latitude"], zval),},
                coords={
                    "longitude": (["longitude"], lon),
                    "latitude": (["latitude"], lat),
                },
                )
    
    lo = ds.coords["longitude"]
    la = ds.coords["latitude"]
    ds["flag"] = xr.full_like(ds.z, fill_value=-100)
    ds["flag"].loc[dict(longitude=lo[(lo > region_lon[0]-5.) 
                                     & (lo < region_lon[1]+5.)], 
                     latitude=la[(la > region_lat[0]-5.) 
                                 & (la < region_lat[1]+5.)])] = 0
    
    
    ds["flag"].loc[dict(longitude=lo[(lo > region_lon[0]) 
                                     & (lo < region_lon[1])], 
                     latitude=la[(la > region_lat[0]) 
                                 & (la < region_lat[1])])] = 100
    
    fval = ds["flag"].values.flatten()
    
    f = open(outfile, "w+")
    with open(outfile, 'w') as f:
        f.write("mshid=3;ellipsoid-grid\n")
        f.write("mdims=2\n")

        f.write("coord=1;%d\n" % (len(lon)))
        f.write("\n".join(map(str, lon)))
        
        f.write("\ncoord=2;%d\n" % (len(lat)))
        f.write("\n".join(map(str, lat)))
        
        f.write("\nvalue=%d;1\n" % (len(fval)))
        f.write("\n".join(map(str, fval)))

    f.close()

    return ds

Let's call the function:

In [5]:
regfile = os.path.join(input_path, "topo.msh")
ds = buildRegMesh(gom_lon, gom_lat, regfile)

### Building the unstructured grid

We will now use `jigsaw` to generate the unstructured mesh from our regular grid...

To do so we define a new function `unstructuredMesh` which takes 3 arguments. The first 2 specify the created `jigsaw` regular file name and directory. The last one `hfn` defines the space conditions for the jigsaw algorithm and consists of 3 values:

+ the spacing in km for the unstructured mesh where the regular grid input defined previously is equal to -100,
+ the spacing in km for the unstructured mesh where the regular grid input defined previously is equal to 0, + the spacing in km for region set to 100.

In [6]:
def unstructuredMesh(dst_path, regfile, hfn):
    '''
    Unstructured mesh generated with `jigsaw` from a regular grid defined with values set to -100,0 and 100.
    
    :arg dst_path: name of the folder containing the `jigsaw` files same as the regular grid one
    :arg regfile: name of the regular grid defined in the previous function
    :arg hfn: space conditions for the jigsaw algorithm
    
    '''

    meshfile = os.path.join(dst_path, "mesh.msh")
    spacefile = os.path.join(dst_path, "spac.msh")
    outfile = os.path.join(dst_path, "mesh.vtk")

    t0 = process_time()
    opts = jigsawpy.jigsaw_jig_t()
    topo = jigsawpy.jigsaw_msh_t()
    geom = jigsawpy.jigsaw_msh_t()
    mesh = jigsawpy.jigsaw_msh_t()
    hmat = jigsawpy.jigsaw_msh_t()

    jigsawpy.loadmsh(regfile, topo)
    print("Load topography grid (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    opts.geom_file = os.path.join(dst_path, "topology.msh")
    opts.jcfg_file = os.path.join(dst_path, "config.jig")
    opts.mesh_file = meshfile
    opts.hfun_file = spacefile

    geom.mshID = "ellipsoid-mesh"
    geom.radii = np.full(3, 6.371e003, dtype=geom.REALS_t)
    jigsawpy.savemsh(opts.geom_file, geom)

    hmat.mshID = "ellipsoid-grid"
    hmat.radii = geom.radii
    hmat.xgrid = topo.xgrid * np.pi / 180.0
    hmat.ygrid = topo.ygrid * np.pi / 180.0

    # Set HFUN gradient-limiter
    hmat.value = np.full(topo.value.shape, hfn[0], dtype=hmat.REALS_t)
    hmat.value[topo.value == 0] = hfn[1]
    hmat.value[topo.value > 10] = hfn[2]

    hmat.slope = np.full(topo.value.shape, +0.050, dtype=hmat.REALS_t)
    jigsawpy.savemsh(opts.hfun_file, hmat)
    jigsawpy.cmd.marche(opts, hmat)
    print("Build space function (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    opts.hfun_scal = "absolute"
    opts.hfun_hmax = float("inf")  # null HFUN limits
    opts.hfun_hmin = float(+0.00)

    opts.mesh_dims = +2  # 2-dim. simplexes

    opts.optm_qlim = +9.5e-01  # tighter opt. tol
    opts.optm_iter = +32
    opts.optm_qtol = +1.0e-05

    jigsawpy.cmd.tetris(opts, 3, mesh)
    print("Perform triangulation (%0.02f seconds)" % (process_time() - t0))

    t0 = process_time()
    apos = jigsawpy.R3toS2(geom.radii, mesh.point["coord"][:])

    apos = apos * 180.0 / np.pi

    zfun = interpolate.RectBivariateSpline(topo.ygrid, topo.xgrid, topo.value)

    mesh.value = zfun(apos[:, 1], apos[:, 0], grid=False)

    jigsawpy.savevtk(outfile, mesh)
    jigsawpy.savemsh(opts.mesh_file, mesh)
    print("Get unstructured mesh (%0.02f seconds)" % (process_time() - t0))

    return

Let's define the resolution for our unstructured mesh. 

Here, we will chose a coarse resolution of 150000 km outside of the region of interest, 100 km around the region and 25 km within the region.

In [7]:
hfn = np.zeros(3)
hfn[0] = 150000.
hfn[1] = 100.
hfn[2] = float(res_mesh)

print('Chosen resolution in km',hfn)

Chosen resolution in km [1.5e+05 1.0e+02 2.5e+01]


We now call `jigsaw` meshing function... 

:::{caution}

This function might takes some times if you choose a fine resolution (<10 km) so be careful!

:::

In [ ]:
unstructuredMesh(input_path, regfile, hfn)

We will load the mesh properties in our jupyter notebook:

+ coordinates of each vertice
+ cells defining the connectivities

In [8]:
meshfile = os.path.join(input_path, "mesh.vtk")

umesh = meshio.read(meshfile)

coords = umesh.points
coords = (coords / 6.371e003) * 6378137.0
cells = umesh.cells_dict["triangle"]

Let's visualise the mesh. After running the cell below you will be able to use the top left widget on the graph to select the `surface with edges` as shown below.


```{figure} images/selectedges.png
:height: 100px
:name: figure-example

`pyvista` selecting edges
```

In [13]:
meshfile = os.path.join(input_path, "mesh.vtk")
mesh = pv.read(meshfile)

plotter = pv.PlotterITK()
plotter.add_mesh(mesh, scalars="value")

plotter.show()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Mapping elevations 

Now that we have our unstructured mesh, we will interpolate the required variables on the new mesh. 
We will start with the elevation. 

:::{note}

As most of the available dataset are defined in lon/lat coordinates, we first define a function `xyz2lonlat` to perform the conversion between cartesian points of the spherical triangulation to lat/lon.

:::

In [14]:
def xyz2lonlat(coords, radius=6378137.0):
    """
    Convert x,y,z representation of cartesian points of the
    spherical triangulation to lat/lon.
    """

    gLonLat = np.zeros((len(coords), 2))

    gLonLat[:, 1] = np.arcsin(coords[:, 2] / radius)
    gLonLat[:, 0] = np.arctan2(coords[:, 1], coords[:, 0])
    gLonLat[:, 1] = np.mod(np.degrees(gLonLat[:, 1]) + 90, 180.0)
    gLonLat[:, 0] = np.mod(np.degrees(gLonLat[:, 0]) + 180.0, 360.0)

    return gLonLat

In [15]:
lonlat = xyz2lonlat(coords, radius=6378137.0)

Here we chose to use the ETOPO5 topography file, obviously other elevation grids could be used. 

The `netcdf` file for this specific dataset can be accessed via `THREDDS` protocol by specifying the related `url`

In [16]:
# Input dataset files

# etopo2
#infile = 'https://data.pmel.noaa.gov/pmel/thredds/dodsC/data/PMEL/smith_sandwell_topo_v8_2.nc'

# etopo5 url
infile = 'http://ferret.pmel.noaa.gov/pmel/thredds/dodsC/data/PMEL/etopo5.nc'

### Transformation with `xarray`

We will use `xarray` library to open the file:

In [18]:
# Read topo
data = xr.open_dataset(infile)
data

<xarray.Dataset>
Dimensions:    (ETOPO05_X: 4320, ETOPO05_Y: 2161)
Coordinates:
  * ETOPO05_X  (ETOPO05_X) float64 0.0 0.08333 0.1667 0.25 ... 359.8 359.8 359.9
  * ETOPO05_Y  (ETOPO05_Y) float64 -90.0 -89.92 -89.83 ... 89.83 89.92 90.0
Data variables:
    ROSE       (ETOPO05_Y, ETOPO05_X) float32 ...
Attributes:
    history:        FERRET V5.22   27-Apr-01, from IRI/LDEO worldbath.nc
    IRI_LDEO_note:  updated 27 Feb 1998 from NGDC CD-ROM 29 April 1993

We will first regrid the longitude (`ETOPO05_X`) to match with our `jigsaw` unstructured mesh (*i.e* between -180 and 180 instead of 0 to 360)

In [19]:
data.coords['ETOPO05_X'] = (data.coords['ETOPO05_X'] + 180) % 360 - 180
data = data.sortby(data.ETOPO05_X)
data

<xarray.Dataset>
Dimensions:    (ETOPO05_X: 4320, ETOPO05_Y: 2161)
Coordinates:
  * ETOPO05_X  (ETOPO05_X) float64 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * ETOPO05_Y  (ETOPO05_Y) float64 -90.0 -89.92 -89.83 ... 89.83 89.92 90.0
Data variables:
    ROSE       (ETOPO05_Y, ETOPO05_X) float32 ...
Attributes:
    history:        FERRET V5.22   27-Apr-01, from IRI/LDEO worldbath.nc
    IRI_LDEO_note:  updated 27 Feb 1998 from NGDC CD-ROM 29 April 1993

We will now only take into account the elevation within our specified region 

In [20]:
# We define a new variable newz
data = data.assign(newz=data["ROSE"])

# We mask all values not in the specified region
mask = ((data.coords["ETOPO05_Y"] > gom_lat[0]) & (data.coords["ETOPO05_Y"] < gom_lat[1])
        & (data.coords["ETOPO05_X"] > gom_lon[0]) & (data.coords["ETOPO05_X"] < gom_lon[1]))

# We set all values not in the specified region with an elevation of -10000.
data["newz"] = xr.where(np.logical_not(mask), -10000, data["newz"])
dataArray = data["newz"].values.T

### Interpolate on the unstructured mesh

Once the elevation array has been built, we now interpolate from the regular dataset to the unstructured mesh:

In [21]:
# We can apply a smoothing on the dataset if needed...
dataArray = ndimage.gaussian_filter(dataArray, sigma=0.1)

# Map regular mesh lon/lat 
ilons = dataArray.shape[0] * lonlat[:, 0] / float(dataArray.shape[0])
ilats = dataArray.shape[1] * lonlat[:, 1] / float(dataArray.shape[1])

# Create the regular grid coordinates and store it as rcoords
icoords = np.stack((ilons, ilats))
rlons = icoords[0, :] * dataArray.shape[0] / 360.0
rlats = icoords[1, :] * dataArray.shape[1] / 180.0

rcoords = np.zeros(icoords.shape)
rcoords[0, :] = rlons
rcoords[1, :] = rlats

Interpolate the elevations on the global unstructured mesh:

In [26]:
meshz = ndimage.map_coordinates(dataArray, rcoords, order=2, 
                                mode="nearest").astype(float)

### Save elevation grid on disk

We now build `gospl` unstructured mesh input for the elevation data which needs to be a `Numpy` compressed file containing:

+ mesh coordinates, 
+ cells, 
+ each vertice neighbours indices and 
+ the nodes elevation,

In [27]:
# Set meshplex triangular mesh
Gmesh = meshplex.MeshTri(coords, cells)
s = Gmesh.idx_hierarchy.shape
a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
Gmesh.edges = {"points": np.unique(a, axis=0)}

# Get each vertice neighbours indices
ngbNbs, ngbID = definegtin(len(coords), Gmesh.cells["points"], 
                           Gmesh.edges["points"])

Store the file in an input folder for `gospl` run (here called `gospl_data`):

In [28]:
# gospl input files directory
mesh_path = "gospl_data" 
if not os.path.exists(mesh_path):
    os.makedirs(mesh_path)

# Set elevation file name
elevfname = os.path.join(mesh_path, "mesh"+str(res_mesh)+"km")


# Save the mesh as compressed numpy file for gospl
np.savez_compressed(elevfname, v=coords, c=cells, n=ngbID[:, :8].astype(int), 
                    z=meshz)

## Mapping rainfall

We will now load a precipitation grid and interpolate the dataset on the unstructured mesh.

Here we chose the CPC collection of precipitation data sets (2.5°x2.5°) containing global monthly values since 1979.

````{margin}
```{seealso}
[PSL](https://psl.noaa.gov/data/gridded/help.html#opendap) climate research data resources is useful for finding precipitation map as well as the THREDDS [catalog](https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html).
```
````

The `netcdf` file for our dataset can be accessed via `THREDDS` protocol by specifying the related `url`:

In [49]:
# Input dataset file url
ncfile = 'https://psl.noaa.gov/thredds/dodsC/Datasets/cmap/enh/precip.mon.mean.nc'

### Transformation with `xarray`

Here again we will use `xarray` to open the file and perform several changes on the dataset:

In [51]:
# Read rain
dr = xr.open_dataset(ncfile)
dr

<xarray.Dataset>
Dimensions:  (lat: 72, lon: 144, time: 506)
Coordinates:
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-02-01
Data variables:
    precip   (time, lat, lon) float32 ...
Attributes:
    Conventions:                     COARDS
    title:                           CPC Merged Analysis of Precipitation (in...
    platform:                        Analyses
    source:                          ftp ftp.cpc.ncep.noaa.gov precip/cmap/mo...
    dataset_title:                   CPC Merged Analysis of Precipitation
    documentation:                   https://www.esrl.noaa.gov/psd/data/gridd...
    date_modified:                   26 Feb 2019
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    version:                         V2103
    history:                         update 03/2021 V2103
    data_modified:                   2021-03-09
    DODS_EXTRA.Unlimited_Dimension:  time

The data consists of average monthly rate of precipitation. `gospl` requires rainfall in `m/yr` we therefore use the `groupby` function to compute the yearly mean.

In [52]:
# Compute annual mean
dryears = dr.groupby('time.year').mean('time')
dryears

<xarray.Dataset>
Dimensions:  (lat: 72, lon: 144, year: 43)
Coordinates:
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * year     (year) int64 1979 1980 1981 1982 1983 ... 2017 2018 2019 2020 2021
Data variables:
    precip   (year, lat, lon) float32 0.5208 0.5142 0.51 0.5042 ... 0.0 0.0 0.0

We now compute the annual mean for the entire dataset running from 1979 to 2021:

In [54]:
dryearly = dryears.mean('year')

We will now interpolate the dataset on a higher resolution grid:

In [55]:
# Set new longitudes for interpolation
new_lon = np.linspace(dryearly.lon[0], dryearly.lon[-1], 
                      dryearly.dims["lon"] * 4)

# Set new latitudes for interpolation
new_lat = np.linspace(dryearly.lat[0], dryearly.lat[-1], 
                      dryearly.dims["lat"] * 4)

# Interpolation
drain = dryearly.interp(lat=new_lat, lon=new_lon)

As for the elevation dataset, we will now regrid the longitude to match with the `jigsaw` unstructured mesh (*i.e* between -180 and 180 instead of 0 to 360)

In [58]:
drain.coords['lon'] = (drain.coords['lon'] + 180) % 360 - 180
drain = drain.sortby(drain.lon).fillna(0)
drain

<xarray.Dataset>
Dimensions:  (lat: 288, lon: 576)
Coordinates:
  * lat      (lat) float64 88.75 88.13 87.51 86.89 ... -87.51 -88.13 -88.75
  * lon      (lon) float64 -179.7 -179.1 -178.4 -177.8 ... 178.4 179.1 179.7
Data variables:
    precip   (lat, lon) float64 0.5698 0.5719 0.5739 ... 0.9475 0.9521 0.957

We now only consider the dataset within our specified region 

In [36]:
# Interpolate
# drain_interp = drain.interp(lon=data['ETOPO05_X'].values, lat=data['ETOPO05_Y'].values)
# drain_interp = drain_interp.fillna(0)

In [59]:
# We define a new variable rain
drain = drain.assign(rain=drain["precip"])

# We mask all values not in the specified region
mask = ((drain.coords["lat"] > gom_lat[0]) & (drain.coords["lat"] < gom_lat[1])
        & (drain.coords["lon"] > gom_lon[0]) & (drain.coords["lon"] < gom_lon[1]))

drain["rain"] = xr.where(np.logical_not(mask), 0, drain["rain"])
drain

<xarray.Dataset>
Dimensions:  (lat: 288, lon: 576)
Coordinates:
  * lat      (lat) float64 88.75 88.13 87.51 86.89 ... -87.51 -88.13 -88.75
  * lon      (lon) float64 -179.7 -179.1 -178.4 -177.8 ... 178.4 179.1 179.7
Data variables:
    precip   (lat, lon) float64 0.5698 0.5719 0.5739 ... 0.9475 0.9521 0.957
    rain     (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

### Interpolate on the unstructured mesh

Once the rainfall array has been built, we now interpolate from the regular dataset to the unstructured mesh:

In [60]:
# Buld the rain array convertion from mm/day to m/year
rainArray = drain.rain.T*366./1000.
rainArray = np.nan_to_num(rainArray)
rainArray[rainArray<0.] = 0
    
# We can apply a smoothing on the dataset if needed...
rainArray = ndimage.gaussian_filter(rainArray, sigma=2)
rainArray[rainArray<0.] = 0

Interpolate the rainfalls on the global unstructured mesh:

In [61]:
# Interpolate the paleogrid on global mesh
meshr = ndimage.map_coordinates(rainArray, rcoords, order=2,
                                mode="nearest").astype(float)

meshr[meshr<0.] = 0.

### Write precipitation grid on disk

We now build `gospl` precipitation input data as a `Numpy` compressed file with the precipitation values for each mesh vertice.

In [62]:
# Set elevation file name
npzrain = os.path.join(mesh_path, "rain"+str(res_mesh)+"km")

# Save the mesh as compressed numpy file for gospl
np.savez_compressed(npzrain, r=meshr)

## Save inputs as a `vtk` file

Before going further, you can check the mesh and interpolated dataset by building a `VTK` file: 

In [64]:
paleovtk = elevfname + ".vtk"

# Define mesh
vis_mesh = meshio.Mesh(coords, {"triangle": cells}, 
                       point_data={"elev": meshz, "precip": meshr})

# Write it disk
meshio.write(paleovtk, vis_mesh)

print("Writing VTK input file as {}".format(paleovtk))

Writing VTK input file as gospl_data/mesh25km.vtk


Let’s visualise the mesh with `pyvista` library:

In [48]:
mesh = pv.read(paleovtk)
elev = mesh.get_array(name='elev')

earthRadius = 6.371e6
scale = 10.
factor = 1.+ (elev/earthRadius)*scale

mesh.points[:, 0] *= factor
mesh.points[:, 1] *= factor
mesh.points[:, 2] *= factor

contour = mesh.contour([0])

plotter = pv.PlotterITK()
plotter.add_mesh(mesh, scalars="elev")
plotter.add_mesh(contour, color="black", opacity=1.)

plotter.show()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…